# Advertising Analytics Click Prediction: ML
####[Ad impressions with clicks dataset](https://www.kaggle.com/c/avazu-ctr-prediction/data)

This is the ML notebook for the series of Advertising Analytics Click Prediction notebooks.  For this stage, I will focus on creating features and training and evaluating the ML model.

In [0]:
import databricks.koalas as ks

In [0]:
data_path = "dbfs:/FileStore/tables/filtered_train.csv"
impression = spark.read.format("csv").option("header", "true").load(data_path).selectExpr("*", "substr(hour, 7) as hr").repartition(64)
impression = impression.drop("_c0")
# impression = spark.read \
#   .parquet("/FileStore/tables/filtered_train.csv/") \
#   .selectExpr("*", "substr(hour, 7) as hr").repartition(64)

In [0]:
display(impression)

In [0]:
#impression.dtypes

In [0]:
# impression['click'] = impression['click'].astype('int')
# impression['banner_pos'] = impression['banner_pos'].astype('int')
# impression['C1'] = impression['C1'].astype('int')
# impression['_c0'] = impression['_c0'].astype('int')
# impression['hour'] = impression['hour'].astype('int')
# impression['id'] = impression['id'].astype('string')
#dict_columns = impression.columns

#impression['id'] = impression['id'].astype('decimal')
#Prepare features
FEATURES_COL = ['click','hour','C1','banner_pos','device_type','device_conn_type','C14','C15','C16','C17','C18','C19','C20','C21','hr']
for col in impression.columns:
    if col in FEATURES_COL:
        impression = impression.withColumn(col,impression[col].cast('int'))
#impression[['click','banner_pos','C1']]= impression[['click','banner_pos','C1']].apply(ks.to_numeric)

# dict_columns_type = {'_c0':int, 'id': string, 
#                  'click':int, 
#                  'hour':int, 
#                  'C1':int, 
#                  'banner_pos':int, 
#                  'site_id':string,
#                  'site_domain':string, 
#                  'site_category':string, 
#                  'app_id':string, 
#                  'app_domain':string, 
#                  'app_category':string,
#                  'device_id':string, 
#                  'device_ip':string, 
#                  'device_model':string, 
#                  'device_type':int,
#                  'device_conn_type':int, 
#                  'C14':int, 
#                  'C15':int, 
#                  'C16':int, 
#                  'C17':int, 
#                  'C18':int, 
#                  'C19':int, 
#                  'C20':int,
#                  'C21':int, 
#                  'hr':int
#                }
   
#impression = impression.astype(dict_columns_type)

# dfobj = dfobj.astype(dict_columns_type)
#dict_columns


In [0]:
impression.dtypes

Out[6]: [('id', 'string'),
 ('click', 'int'),
 ('hour', 'int'),
 ('C1', 'int'),
 ('banner_pos', 'int'),
 ('site_id', 'string'),
 ('site_domain', 'string'),
 ('site_category', 'string'),
 ('app_id', 'string'),
 ('app_domain', 'string'),
 ('app_category', 'string'),
 ('device_id', 'string'),
 ('device_ip', 'string'),
 ('device_model', 'string'),
 ('device_type', 'int'),
 ('device_conn_type', 'int'),
 ('C14', 'int'),
 ('C15', 'int'),
 ('C16', 'int'),
 ('C17', 'int'),
 ('C18', 'int'),
 ('C19', 'int'),
 ('C20', 'int'),
 ('C21', 'int'),
 ('hr', 'int')]

In [0]:
# strCols = [item[0] for item in impression.dtypes if item[1].startswith('string')]
# intCols = [item[0] for item in impression.dtypes if item[1].startswith('int')]


In [0]:
from pyspark.sql.functions import *

# strCols = map(lambda t: t[0], map(lambda t: t[1] == 'string', impression.dtypes))
# intCols = map(lambda t: t[0], map(lambda t: t[1] == 'int', impression.dtypes))

# # [row_idx][json_idx]
# strColsCount = sorted(map(lambda c: (c, impression.select(countDistinct(c)).collect()[0][0]), strCols), key=lambda x: x[1], reverse=True)
# intColsCount = sorted(map(lambda c: (c, impression.select(countDistinct(c)).collect()[0][0]), intCols), key=lambda x: x[1], reverse=True)

strCols = [item[0] for item in impression.dtypes if item[1].startswith('string')]
intCols = [item[0] for item in impression.dtypes if item[1].startswith('int')]

strColsCount = sorted(map(lambda c: (c, impression.select(countDistinct(c)).collect()[0][0]), strCols), key=lambda x: x[1], reverse=True)
intColsCount = sorted(map(lambda c: (c, impression.select(countDistinct(c)).collect()[0][0]), intCols), key=lambda x: x[1], reverse=True)

In [0]:
# distinct counts for str columns
display(strColsCount)

_1,_2
id,402586
device_ip,261706
device_id,64742
device_model,4380
app_id,2241
site_id,2225
site_domain,2188
app_domain,143
app_category,27
site_category,22


In [0]:
# distinct counts for int columns
display(intColsCount)

_1,_2
C14,2088
C17,411
hour,240
C20,161
C19,65
C21,60
hr,24
C16,9
C15,8
C1,7


In [0]:
# Include PySpark Feature Engineering methods
from pyspark.ml.feature import StringIndexer, VectorAssembler

# All of the columns (string or integer) are categorical columns
#  except for the [click] column
maxBins = 70
categorical = [item[0] for item in strColsCount if item[1] <= maxBins]
categorical += [item[0] for item in intColsCount if item[1] <= maxBins]
#map(lambda c: c[0], filter(lambda c: c[1] <= maxBins, intColsCount))
# categorical = map(lambda c: c[0], filter(lambda c: c[1] <= maxBins, strColsCount))
# categorical += map(lambda c: c[0], filter(lambda c: c[1] <= maxBins, intColsCount))
categorical.remove('click')

# Apply string indexer to all of the categorical columns
#  And add _idx to the column name to indicate the index of the categorical value
#stringIndexers = map(lambda c: StringIndexer(inputCol = c, outputCol = c + "_idx"), categorical)
#stringIndexers = [item + "_idx" for item in categorical]
stringIndexers = [StringIndexer(inputCol = item, outputCol = item + "_idx") for item in categorical] 

# Assemble the put as the input to the VectorAssembler 
#   with the output being our features
# assemblerInputs = map(lambda c: c + "_idx", categorical)
assemblerInputs = [item + "_idx" for item in categorical]
vectorAssembler = VectorAssembler(inputCols = assemblerInputs, outputCol = "features")

# # The [click] column is our label 
labelStringIndexer = StringIndexer(inputCol = "click", outputCol = "label")

# # The stages of our ML pipeline 
stages = stringIndexers + [vectorAssembler, labelStringIndexer]

#stringIndexers = [StringIndexer(inputCol = item, outputCol = item + "_idx") for item in categorical] 

In [0]:
from pyspark.ml import Pipeline

# Create our pipeline
pipeline = Pipeline(stages = stages)

# create transformer to add features
featurizer = pipeline.fit(impression)

In [0]:
# dataframe with feature and intermediate transformation columns appended
featurizedImpressions = featurizer.transform(impression)

In [0]:
display(featurizedImpressions.select('features', 'label'))

In [0]:
train, test = featurizedImpressions \
  .select(["label", "features", "hr"]) \
  .randomSplit([0.7, 0.3], 42)
train.cache()
test.cache()

Out[15]: DataFrame[label: double, features: vector, hr: int]

In [0]:
from pyspark.ml.classification import GBTClassifier

# Train your model !
# Train our GBTClassifier model
classifier = GBTClassifier(labelCol="label",
featuresCol="features", maxBins=maxBins, maxDepth=10, maxIter=10)
model = classifier.fit(train)


In [0]:
# Execute our predictions
predictions = model.transform(test)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate our GBTClassifier model using BinaryClassificationEvaluator()
ev = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")
print(ev.evaluate(predictions))

0.7045708156320244


In [0]:
#predictions.schema['features'].metadata.get('ml_attr').get('attrs').values()[0]
list(predictions.schema['features'].metadata.get('ml_attr').get('attrs').values())[0]

Out[19]: [{'vals': ['07d7df22',
   '0f2161f8',
   'cef3e649',
   '8ded1f7a',
   'f95efa07',
   'd1327cf5',
   'dc97ec06',
   '09481d60',
   '75d80bbe',
   'fc6fa53d',
   '4ce2e9fc',
   '879c24eb',
   'a3c42688',
   '0f9a328c',
   '4681bb9d',
   '8df2e842',
   'a86a3e89',
   '2281a340',
   '79f0b860',
   '0bfbc358',
   '5326cf99',
   '0d82db25',
   '18b1e0be',
   '2fc4f2aa',
   '7113d72a',
   'a7fd01ec',
   '4b7ade46'],
  'idx': 0,
  'name': 'app_category_idx'},
 {'vals': ['50e219e0',
   'f028772b',
   '28905ebd',
   '3e814130',
   'f66779e6',
   '75fa27f6',
   '335d28a8',
   '76b2941d',
   'c0dd3be3',
   '72722551',
   'dedf689d',
   '70fb0e29',
   '0569f928',
   '8fd0aea4',
   '42a36e14',
   'a818d37a',
   'bcf865d9',
   'e787de0e',
   '5378d028',
   '9ccfa2ea',
   'a72a0145',
   'c706e647'],
  'idx': 1,
  'name': 'site_category_idx'},
 {'vals': ['35',
   '39',
   '167',
   '161',
   '47',
   '1327',
   '297',
   '163',
   '175',
   '679',
   '935',
   '687',
   '1063',
   '41',
   '3

In [0]:
import json
features = map(lambda c: str(json.loads(json.dumps(c))['name']),list(predictions.schema['features'].metadata.get('ml_attr').get('attrs').values())[0])
# convert numpy.float64 to str for spark.createDataFrame()
weights=map(lambda w: '%.10f' % w, model.featureImportances)
weightedFeatures = sorted(zip(weights, features), key=lambda x: x[1], reverse=True)
spark.createDataFrame(weightedFeatures).toDF("weight", "feature").createOrReplaceTempView('wf')

In [0]:
%sql 
select feature, weight 
from wf 
order by weight desc

feature,weight
C21_idx,0.2992599531
hr_idx,0.2295530900
C19_idx,0.1233527078
site_category_idx,0.0841000544
app_category_idx,0.0806769299
C16_idx,0.0554613087
device_conn_type_idx,0.0353353315
banner_pos_idx,0.0288683588
C18_idx,0.0257540325
C1_idx,0.0199385091


In [0]:
predictions.createOrReplaceTempView("predictions")

In [0]:
%sql describe predictions

col_name,data_type,comment
label,double,null
features,vector,null
hr,int,null
rawPrediction,vector,null
probability,vector,null
prediction,double,null


In [0]:
%sql select sum(case when prediction = label then 1 else 0 end) / (count(1) * 1.0) as accuracy
from predictions

accuracy
0.83279477710366658
